<a href="https://colab.research.google.com/github/ShriMLEngineer/furniture_classification/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn torch torchvision Pillow python-multipart

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 2.2 MB/s eta 0:00:00
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31678 sha256=09503a123c0a9384f6c4da5dc58924b2db07661a158eb55c645955c3a59b3ffd
  Stored in directory: /root/.cache/pip/wheels/9e/fc/1c/cf980e6413d3ee8e70cd8f39e2366b0f487e3e221aeb452eb0
Successfully built python-multipart


In [ ]:
from fastapi import FastAPI, File, UploadFile
from PIL import Image
import io
import torch
import torchvision.transforms as T
import asyncio

In [ ]:
app = FastAPI()

path = '/content/furniture_classification/detr-main'

model = torch.hub.load(path, 'detr_resnet50', source="local", pretrained=True, num_classes=4)
checkpoint = torch.load("/content/furniture_classification/outputs/checkpoint.pth", map_location='cpu')

model.load_state_dict(checkpoint['model'], strict=False)
model.eval();

FileNotFoundError: ignored

In [ ]:
@app.post('/predict')
async def predict(file: UploadFile = File(...)):

  contents = await file.read()
  img = Image.open(io.BytesIO(contents)).convert('RGB')
  img = img.resize((224,224))
  img = torch.unsqueeze(T.ToTensor()(img), 0)
  pred = model(img)

  predicted_class = torch.argmax(pred).item()

  return{'class': predicted_class}